In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore' )

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/work/steam-200k.csv')

In [ ]:
 df.head()

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [ ]:
cols = {'151603712':'user_Id','The Elder Scrolls V Skyrim':'game_name','purchase':'status','1.0':'Hourplayed'}
df.rename(columns = cols,inplace =True)

In [ ]:
df.head()

,user_Id,game_name,status,Hourplayed,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [ ]:
df.drop(columns =['0'],inplace =True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_Id     199999 non-null  int64  
 1   game_name   199999 non-null  object 
 2   status      199999 non-null  object 
 3   Hourplayed  199999 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [ ]:
df.duplicated().sum()

707

In [ ]:
# droping a duplicated values:
df.drop_duplicates(inplace=True)

In [ ]:
df.isnull().sum()

user_Id       0
game_name     0
status        0
Hourplayed    0
dtype: int64

In [ ]:
df.nunique()

user_Id       12393
game_name      5155
status            2
Hourplayed     1593
dtype: int64

Converting hours into ratings

In [ ]:
# from this we get id of player how had played a game for more than or equal to 2 hours
df= df[(df['Hourplayed']>=2) & (df['status']=='play')]

In [ ]:
df = df[df.groupby('game_name').user_Id.transform(len)>=20]

In [ ]:
df['game_name'].nunique()

431

In [ ]:
df['game_name'] = df['game_name'].astype(str)

In [ ]:
average = df.groupby(['game_name'],as_index = False).Hourplayed.mean()

In [ ]:
average['avg_hourplayed'] = average['Hourplayed']
average.drop(columns ='Hourplayed',inplace = True )

In [ ]:
average.head()

,game_name,avg_hourplayed
0,7 Days to Die,50.777500
1,APB Reloaded,113.377778
2,ARK Survival Evolved,105.982812
3,Ace of Spades,10.971739
4,AdVenture Capitalist,67.760674


In [ ]:
df = df.merge(average,on = 'game_name')

In [ ]:
condition = [
    df['Hourplayed']>= (0.8*df['avg_hourplayed']),
   (df['Hourplayed']>=0.6*df['avg_hourplayed'])&(df['Hourplayed']<0.8*df['avg_hourplayed']),
   (df['Hourplayed']>=0.4*df['avg_hourplayed'])&(df['Hourplayed']<0.6*df['avg_hourplayed']),
   (df['Hourplayed']>=0.2*df['avg_hourplayed'])&(df['Hourplayed']<0.4*df['avg_hourplayed']),
    df['Hourplayed']>=0
    
]
values = [5,4,3,2,1]
df['rating'] = np.select(condition,values)

In [ ]:
df.head()

,user_Id,game_name,status,Hourplayed,avg_hourplayed,rating
0,151603712,The Elder Scrolls V Skyrim,play,273.0,115.351792,5
1,59945701,The Elder Scrolls V Skyrim,play,58.0,115.351792,3
2,92107940,The Elder Scrolls V Skyrim,play,110.0,115.351792,5
3,250006052,The Elder Scrolls V Skyrim,play,465.0,115.351792,5
4,11373749,The Elder Scrolls V Skyrim,play,220.0,115.351792,5


In [ ]:
# keeping all important columns (user_id,game_name,rating ) and drop all other columns
df.drop(columns = [ 'status', 'Hourplayed', 'avg_hourplayed'],inplace =True )

In [ ]:
df.head()

,user_Id,game_name,rating
0,151603712,The Elder Scrolls V Skyrim,5
1,59945701,The Elder Scrolls V Skyrim,3
2,92107940,The Elder Scrolls V Skyrim,5
3,250006052,The Elder Scrolls V Skyrim,5
4,11373749,The Elder Scrolls V Skyrim,5


In [ ]:
ratings=df

In [ ]:
from sklearn import preprocessing

In [ ]:
Y = ratings['game_name']
le =preprocessing.LabelEncoder()
ratings['game_id'] = le.fit_transform(Y)

In [ ]:
ratings.drop(columns = ['game_name'],inplace =True )

In [ ]:
Y = ratings['user_Id']
le =preprocessing.LabelEncoder()
ratings['user_id'] = le.fit_transform(Y)

In [ ]:
ratings.drop(columns = ['user_Id'],inplace =True )

In [ ]:
ratings.head()

,rating,game_id,user_id
0,5,365,4114
1,3,365,1111
2,5,365,2076
3,5,365,7282
4,5,365,135


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36545 entries, 0 to 36544
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   rating   36545 non-null  int64
 1   game_id  36545 non-null  int64
 2   user_id  36545 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


In [ ]:
np.random.seed(123)

rand_user_ids = np.random.choice(ratings['user_id'].unique(), 
                                size=int(len(ratings['user_id'].unique())*0.1), 
                                replace=False)

ratings = ratings.loc[ratings['user_id'].isin(rand_user_ids)]

In [ ]:
ratings['rank_latest'] = ratings.groupby('user_id').count()['rating']>150

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = ratings[['user_id', 'game_id', 'rating']]
test_ratings = ratings[['user_id', 'game_id', 'rating']]

In [ ]:
train_ratings.loc[:, 'rating'] = 1

In [ ]:
# Get a list of all movie IDs
all_game_ids = ratings['game_id'].unique()

# Placeholders that will hold the training data
users, games, labels = [], [], []

# This is the set of items that each user has interaction with
users_item_set = set(zip(train_ratings['user_id'], train_ratings['game_id']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in users_item_set:
    users.append(u)
    games.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_game_ids) 
        # check that the user has not interacted with this item
        while (u, negative_item) in users_item_set:
            negative_item = np.random.choice(all_game_ids)
        users.append(u)
        games.append(negative_item)
        labels.append(0) # items not interacted with are negative

In [ ]:
import torch
from torch.utils.data import Dataset

class GamePredictionTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_game_ids):
        self.users, self.games, self.labels = self.get_dataset(ratings, all_game_ids)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.games[idx], self.labels[idx]

    def get_dataset(self, ratings, all_game_ids):
        users, games, labels = [], [], []
        users_item_set = set(zip(ratings['user_id'], ratings['game_id']))

        num_negatives = 4
        for u, i in users_item_set:
            users.append(u)
            games.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_game_ids)
                while (u, negative_item) in users_item_set:
                    negative_item = np.random.choice(all_game_ids)
                users.append(u)
                games.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(games), torch.tensor(labels)

In [ ]:
!pip install pytorch_lightning -qqq


     |████████████████████████████████| 707 kB 28.4 MB/s 
     |████████████████████████████████| 5.9 MB 53.2 MB/s 
     |████████████████████████████████| 419 kB 51.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu

Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 1.7 MB 47.7 MB/s 
     |████████████████████████████████| 438 kB 66.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


In [ ]:
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_games, ratings, all_game_ids):
        super().__init__()
        self.users_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.games_embedding = nn.Embedding(num_embeddings=num_games, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=64)
        self.output = nn.Linear(in_features=64, out_features=1)
        self.ratings = ratings
        self.all_game_ids = all_game_ids
        
    def forward(self, users_input, games_input):
        
        # Pass through embedding layers
        users_embedded = self.users_embedding(users_input)
        games_embedded = self.games_embedding(games_input)

        # Concat the two embedding layers
        vector = torch.cat([users_embedded, games_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))
        vector = nn.ReLU()(self.fc3(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        users_input, games_input, labels = batch
        predicted_labels = self(users_input, games_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(GamePredictionTrainDataset(self.ratings, self.all_game_ids),
                          batch_size=512, num_workers=4)

In [ ]:
num_users = ratings['user_id'].max()+1
num_games = ratings['game_id'].max()+1
all_game_ids = ratings['game_id'].unique()

model = NCF(num_users, num_games, train_ratings, all_game_ids)

trainer = pl.Trainer(max_epochs=3, gpus=1)

trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type      | Params
----------------------------------------------
0 | users_embedding | Embedding | 66.5 K
1 | games_embedding | Embedding | 3.4 K 
2 | fc1             | L

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['user_id'], test_ratings['game_id']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('user_id')['game_id'].apply(list).to_dict()

hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_game_ids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 10 is 0.24
